In [18]:
# Python Libraries
import random
import math
import numbers
import platform
import copy
import os
import time
import pickle

# Importing essential libraries for basic image manipulations.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms
import torchvision.transforms.functional as tF
import torchvision.models as models

In [19]:
%matplotlib inline

# Enable/Disable GPU 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Create custom dataset

In [56]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, test=False):
        img_labels = pd.read_csv(annotations_file, header = 0)
        if test == 1:
            self.img_labels = img_labels.iloc[round(0.9 * img_labels.shape[0]):] 
        elif test == 0:
            self.img_labels = img_labels.iloc[:round(0.9 * img_labels.shape[0])]
        else:
            self.img_labels = img_labels
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image.float(), label

In [21]:
batch_size = 4

transform = transforms.Compose(
    [transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomHorizontalFlip(p=0.5)])

trainset = CustomImageDataset(annotations_file = './data/annotation_file.csv', 
                              img_dir='./data/champion-classifier', transform=transform, test=False)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=0, shuffle=True)

testset = CustomImageDataset(annotations_file = './data/annotation_file.csv', 
                              img_dir='./data/champion-classifier', transform=None, test=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=0, shuffle=True)

### resNet18

In [22]:
class Champion_Net(nn.Module):
    def __init__(self, num_classes, criterion):
        super(Champion_Net, self).__init__()

        # Implement me
        model_ft = models.resnet18()
        model_ft.fc = nn.Linear(512, num_classes)
        
        self.model = model_ft
        self.criterion = criterion
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inp):
        
        pred = self.model(inp)
        return pred

In [23]:
def train_model(model, dataloaders, optimizer, num_epochs=25):
    
    since = time.time()
    acc_list = []
    model.train() # In training mode

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # backward + optimize only if in training phase
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Epoch information
        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        acc_list.append(epoch_acc)

        print('Training Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return acc_list

In [24]:
def eval_model(model, dataloaders):
    
    since = time.time()
    model.eval() # In training mode

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloaders:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    overall_loss = running_loss / len(dataloaders.dataset)
    overall_acc = running_corrects.double() / len(dataloaders.dataset)

    print('Evaluation Loss: {:.4f} Acc: {:.4f}'.format(overall_loss, overall_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return overall_acc

In [30]:
def vec2Champ(vec):

    with open("data/master_champ_list.pkl", "rb") as input_file:
        master_champ_list = pickle.load(input_file)
        
    vec2Champ = {i : champ for i, champ in enumerate(master_champ_list)}
    champ = [vec2Champ.get(i) for i in vec]
    
    return champ

In [38]:
def model_pred(model, dataloaders):
    
    since = time.time()
    model.eval() # In prediction mode

    labels = []

    # Iterate over data.
    for inputs, _ in dataloaders:
        inputs = inputs.to(device)
        
        with torch.no_grad():

            # forward
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            labels += preds.tolist()


    time_elapsed = time.time() - since
    print('Prediction complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return labels, vec2Champ(labels)

### Training

In [26]:
champion_classifer = Champion_Net(86, nn.CrossEntropyLoss()).to(device)

In [27]:
optimizer_SGD = torch.optim.SGD(champion_classifer.parameters(), lr=0.01, momentum=0.9)

In [28]:
train_model(champion_classifer, trainloader, optimizer_SGD, 8)

Epoch 0/7
----------


KeyboardInterrupt: 

### Batch First

In [58]:
# Batch 1
BatchSet_1 = CustomImageDataset(annotations_file = './data/batch-data-0/annotation_file-0.csv', 
                              img_dir='./data/batch-data-0', transform=None, test=2)
BatchSetLoader_1 = torch.utils.data.DataLoader(BatchSet_1, batch_size=10, num_workers=0, shuffle=True)

In [59]:
# Make prediction 
lab_1, champ_1 = model_pred(champion_classifer, BatchSetLoader_1)

Prediction complete in 0m 11s


In [68]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'batch-data-0', 'annotation_file-0.csv'), index_col=False)

df.champion_name = champ_1
df.champion_onehot = lab_1

df.to_csv(os.path.join(os.getcwd(), 'data', 'batch-data-0', 'annotation_file-0.csv'), index = False)


### Save / Load

In [48]:
torch.save(champion_classifer.state_dict(), './model/champion_classifier_bin0.pt')

In [8]:
champion_classifer = Champion_Net(86, nn.CrossEntropyLoss()).to(device)
champion_classifer.load_state_dict(torch.load('./model/champion_classifier_bin0.pt', 
                                              map_location=device))

<All keys matched successfully>

In [13]:
# champion_classifer.eval()
# pred = champion_classifer(inp.float().to(device))

In [14]:
# pred

In [10]:
device

device(type='cpu')